In [ ]:
# pip install yagmail
# pip install imap_tools

In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

#### Работа с почтой

In [ ]:
import yagmail
from imap_tools import MailBox, AND

In [ ]:
# Отправка сообщения
def send_email(to, subject, body):
  yag = yagmail.SMTP(os.environ['EMAIL_LOGIN'], os.environ['EMAIL_PASSWORD'])
  yag.send(to, subject, body)

In [ ]:
send_email('antonio.litvinov@gmail.com', 'Тестовое письмо', 'Тело письма')

In [ ]:
# Получение сообщений
def fetch_emails(limit: int = 10, unread_only: bool = False):
    result = []
    search_criteria = AND(seen=False) if unread_only else AND(all=True)

    with MailBox('imap.gmail.com').login(os.environ['EMAIL_LOGIN'], os.environ['EMAIL_PASSWORD']) as mailbox:
        for msg in mailbox.fetch(search_criteria, limit=limit):
            result.append({
                "from": msg.from_,
                "subject": msg.subject,
                "text": msg.text
            })

    return result

In [ ]:
# Печать сообщений
def print_emails(email_list):
    for msg in email_list:
        print(f"From: {msg['from']}")
        print(f"Subject: {msg['subject']}")
        print("=" * 50)

In [ ]:
print_emails(fetch_emails())

#### Заполнение базы данных фильмов

In [ ]:
import pandas as pd
import psycopg2

In [ ]:
def fill_database(filename: str):
    # Загрузка CSV-файла
    df = pd.read_csv(filename, sep="\t")

    # Подключение к базе данных
    connection = psycopg2.connect(
        user=os.getenv('DB_USER', ''),
        password=os.getenv('DB_PASSWORD', ''),
        host=os.getenv('DB_HOST', ''),
        port=os.getenv('DB_PORT', ''),
        database=os.getenv('DB_NAME', '')
    )
    
    cursor = connection.cursor()

    # Вставка строк в таблицу
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO movies (title, type, year, genre, rating)
            VALUES (%s, %s, %s, %s, %s)
        """, (row['Название'], row['Тип'], row['Год'], row['Жанр'], row['Рейтинг']))

    connection.commit()
    cursor.close()
    connection.close()

#### Получение списка файлов с NAS

In [ ]:
from synology_api import filestation, downloadstation

In [ ]:
#nas.get_info()
#nas.get_list_share()
#nas.get_file_list("/video")

def get_file_list(folder: str):

    user = os.environ['NAS_USER']
    password = os.environ['NAS_PASSWORD']
    host = os.environ['NAS_HOST']
    port = os.environ['NAS_PORT']
    
    nas = filestation.FileStation(host, port, user, password,
                                  secure=False, cert_verify=False, dsm_version=7, debug=True, otp_code=None)
  
    files = nas.get_file_list(folder)
    file_names = [file['name'] for file in files['data']['files']]
    return file_names

In [ ]:
get_file_list('/video')